<a href="https://colab.research.google.com/github/santi2827/test-binder/blob/main/test_web_app4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install voila
#!jupyter serverextension enable --sys-prefix voila

In [2]:
!pip install fastai --upgrade

Requirement already up-to-date: fastai in /usr/local/lib/python3.7/dist-packages (2.4)


In [3]:
from fastai.vision.all import *
from fastai.vision.widgets import *

In [4]:
from fastai.basics import *
from fastai.tabular.core import *
from fastai.tabular.model import *

In [5]:
from fastai.tabular.data import *

In [6]:
# Mount Google Drive
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
class TabularLearner(Learner):
    "`Learner` for tabular data"
    def predict(self, row):
        "Predict on a Pandas Series"
        # dl = self.dls.test_dl(row.to_frame().T)
        dl = self.dls.test_dl(row)
        dl.dataset.conts = dl.dataset.conts.astype(np.float32)
        inp,preds,_,dec_preds = self.get_preds(dl=dl, with_input=True, with_decoded=True)
        b = (*tuplify(inp),*tuplify(dec_preds))
        full_dec = self.dls.decode(b)
        return full_dec,dec_preds[0],preds[0]

In [8]:
@delegates(Learner.__init__)
def tabular_learner(dls, layers=None, emb_szs=None, config=None, n_out=None, y_range=None, **kwargs):
    "Get a `Learner` using `dls`, with `metrics`, including a `TabularModel` created using the remaining params."
    if config is None: config = tabular_config()
    if layers is None: layers = [200,100]
    to = dls.train_ds
    emb_szs = get_emb_sz(dls.train_ds, {} if emb_szs is None else emb_szs)
    if n_out is None: n_out = get_c(dls)
    assert n_out, "`n_out` is not defined, and could not be inferred from data, set `dls.c` or pass `n_out`"
    if y_range is None and 'y_range' in config: y_range = config.pop('y_range')
    model = TabularModel(emb_szs, len(dls.cont_names), n_out, layers, y_range=y_range, **config)
    return TabularLearner(dls, model, **kwargs)

# **Load Learner**

In [9]:
# learn_inf = load_learner(path/'export.pkl', cpu=True)
learn_inf = load_learner('/content/drive/MyDrive/Hairstyle Recommend/export.pkl', cpu=True)
btn_upload = widgets.FileUpload()
btn_run = widgets.Button(description='Classify')
out_pl = widgets.Output()
lbl_pred = widgets.Label()

In [10]:
print(type(learn_inf))

<class '__main__.TabularLearner'>


In [11]:
learn_inf.dls.vocab

['heart', 'long', 'oval', 'round', 'square']

# **Determine Function for Upload Button**

In [12]:
def on_data_change(change):
  """
    lbl_pred.value = ''
    img = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    with out_pl: display(img.to_thumb(128,128))
    pred,pred_idx,probs = learn_inf.predict(img)
    lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'"""

In [13]:
btn_upload.observe(on_data_change, names=['data'])

In [14]:
btn_upload

FileUpload(value={}, description='Upload')

# **Determin Funtion for Classify Button**

In [15]:
def on_click_classify(change):
  """
    img = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    with out_pl: display(img.to_thumb(128,128))
    pred,pred_idx,probs = learn_inf.predict(img)
    lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'"""

In [16]:
#hide_output
btn_run = widgets.Button(description='Classify')

In [17]:
btn_run

Button(description='Classify', style=ButtonStyle())

In [18]:
btn_run.on_click(on_click_classify)

# **The Web Application starts here**

In [19]:

display(VBox([widgets.Label('Select your face!'), btn_upload, btn_run , out_pl, lbl_pred]))

In [20]:
import ipywidgets as widgets

slider = widgets.FloatSlider(description='$x$', value=4)
text = widgets.FloatText(disabled=True, description='$x^2$')

def compute(*ignore):
    text.value = str(slider.value ** 2)

slider.observe(compute, 'value')

widgets.VBox([slider, text])

In [21]:
import pandas as pd

iris = pd.read_csv('https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv')
iris

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica
